In [45]:
from pydantic import BaseModel, Field
from typing import Annotated,Union,Optional
from pydantic import ConfigDict
from typing import Any, Dict, get_origin, get_args
from pydantic.fields import FieldInfo
from typing import List
from datetime import datetime
from deepdiff import DeepDiff

In [46]:

class SimpleUser(BaseModel):
    """A simple user model for testing"""
    name: str
    age: int
    email: str

class ComplexUser(BaseModel):
    """A more complex user model with various field types"""
    model_config = ConfigDict(extra='forbid', str_strip_whitespace=True)
    
    id: int
    name: str = Field(..., min_length=1, max_length=100)
    email: str = Field(..., pattern=r'^[^@]+@[^@]+\.[^@]+$')
    age: Optional[int] = Field(None, ge=0, le=150)
    tags: List[str] = Field(default_factory=list)
    created_at: datetime = Field(default_factory=datetime.now)
    is_active: bool = True

class Product(BaseModel):
    """Product model with nested validation"""
    name: str
    price: float = Field(..., gt=0)
    description: Optional[str] = None
    category: str = "general"

class test(BaseModel):
    a:Annotated[int, Field(gt=0)] = 1
    b:Annotated[Union[str,None], Field(min_length=1)]
    c:Optional[int]

class B(BaseModel):
    a: int = 1
class A(BaseModel):
    a: int = 1

In [47]:
def _type_in_annotation(annotation: Any, target_type: type) -> bool:
        """Check if target_type is already in the annotation"""
        if annotation == target_type or annotation == Any or target_type == Any:
            return True
            
        return get_origin(annotation) is Union and target_type in get_args(annotation)

In [48]:
FieldInfo.from_annotated_attribute(
            annotation=Union[int,str],
            default=None
        )

FieldInfo(annotation=Union[int, str], required=False, default=None)

In [49]:
from functools import reduce
from operator import xor

my_list = [None,1,1,None]

# Reduce the list by XOR
m=map(lambda a:a is not None, my_list)
xor_result = reduce(xor, m)
xor_result

False

In [50]:
complex_type=Union[List[Union[A,int,B]],Dict[str,Union[A,int,B]],None]
u1= Union[Any,int,str,None]

In [51]:
f1=FieldInfo.from_annotated_attribute(
            annotation=A,
            default=Field(default_factory=lambda:1)
        )
an=f1.annotation
get_origin(an)

In [52]:
class Tag:
    pass
# Create a test model with various field types
class TestModel(BaseModel):
    required_field: str
    optional_field: Optional[str] = None
    default_field: str = "default"
    factory_field: List[str] = Field(default_factory=list)
    constrained_field: str = Field(min_length=1, max_length=10, description="A constrained field")
    aliased_field: int = Field(alias="aliased_name", ge=0)
    anf:Annotated[int|float,Tag(),Field(default_factory=lambda: 42)]

# Inspect the model fields
for name, field_info in TestModel.model_fields.items():
    print(f"\n{name}:")
    print(f"  annotation: {field_info.annotation}")
    print(f"  default: {field_info.default}")
    print(f"  is_required: {field_info.is_required()}")
    print(f"  alias: {field_info.alias}")
    print(f"  description: {field_info.description}")


required_field:
  annotation: <class 'str'>
  default: PydanticUndefined
  is_required: True
  alias: None
  description: None

optional_field:
  annotation: typing.Optional[str]
  default: None
  is_required: False
  alias: None
  description: None

default_field:
  annotation: <class 'str'>
  default: default
  is_required: False
  alias: None
  description: None

factory_field:
  annotation: typing.List[str]
  default: PydanticUndefined
  is_required: False
  alias: None
  description: None

constrained_field:
  annotation: <class 'str'>
  default: PydanticUndefined
  is_required: True
  alias: None
  description: A constrained field

aliased_field:
  annotation: <class 'int'>
  default: PydanticUndefined
  is_required: True
  alias: aliased_name
  description: None

anf:
  annotation: int | float
  default: PydanticUndefined
  is_required: False
  alias: None
  description: None


In [53]:
def print_field_info(original_field):
    print("FieldInfo attributes:")
    for attr in dir(original_field):
        if not attr.startswith('_') and hasattr(original_field, attr):
            try:
                value = getattr(original_field, attr)
                if not callable(value):
                    print(f"  {attr}: {value}")
            except:
                pass

In [54]:
DeepDiff(original_field,new_field)

{'type_changes': {'root.description': {'old_type': str,
   'new_type': NoneType,
   'old_value': 'Constrained',
   'new_value': None},
  'root._original_assignment': {'old_type': pydantic.fields.FieldInfo,
   'new_type': pydantic_core._pydantic_core.PydanticUndefinedType,
   'old_value': FieldInfo(annotation=NoneType, required=True, description='Constrained', metadata=[MinLen(min_length=1), MaxLen(max_length=10)]),
   'new_value': PydanticUndefined}},
 'values_changed': {'root._attributes_set': {'new_value': {'annotation': str,
    'frozen': None},
   'old_value': {'description': 'Constrained',
    'min_length': 1,
    'max_length': 10,
    'annotation': str}}},
 'iterable_item_removed': {'root.metadata[0]': MinLen(min_length=1),
  'root.metadata[1]': MaxLen(max_length=10)}}

In [55]:
FieldInfo(**dir(original))

TypeError: pydantic.fields.FieldInfo() argument after ** must be a mapping, not list

In [ ]:
type(original.alias)

NoneType

In [ ]:
original._attributes_set  

{'default': 'demo',
 'description': 'Sample field for demo',
 'min_length': 1,
 'max_length': 50,
 'annotation': str}

In [57]:
field_with_meta=TestModel.model_fields['anf']

In [60]:
field_with_meta

FieldInfo(annotation=Union[int, float], required=False, default_factory=<lambda>, metadata=[<__main__.Tag object at 0x00000284CB545F10>])

In [62]:
def copy_with_construct(original: FieldInfo, **overrides) -> FieldInfo:
    """Use _construct to properly merge FieldInfo instances"""
    return FieldInfo._construct(
        metadata=original,  # Pass original as metadata to merge
        **overrides
    )

In [61]:
copy_with_construct(field_with_meta,annotation=str)

AttributeError: type object 'FieldInfo' has no attribute '_construct'

In [67]:
from field_ops import modify_fieldinfo

In [71]:
m1=modify_fieldinfo(field_with_meta, annotation=str, default=42,)

In [ ]:
DeepDiff(field_with_meta,m1)

{'type_changes': {'root.annotation': {'old_type': types.UnionType,
   'new_type': str,
   'old_value': int | float,
   'new_value': str},
  'root.default': {'old_type': pydantic_core._pydantic_core.PydanticUndefinedType,
   'new_type': int,
   'old_value': PydanticUndefined,
   'new_value': 42}}}

In [75]:
Union[int,str]== Union[str,int]

True